# Simple Neighbourhood Approach (manually implementing user-based CF)
As a first step, we will use basic neighbourhood-based collaborative filtering (CF) techniques, with a simple model as a baseline.

### Pre-Processing

In [90]:
%%capture
import scipy as sp
import scipy.stats as stats
import powerlaw as pl
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import duckdb as db
import recbole as rb
import scipy as sp
import surprise as sks
import sklearn as sk
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [91]:
# Download latest version of data
path = kagglehub.dataset_download("rdoume/beerreviews", path='beer_reviews.csv', force_download = True)
beer = pd.read_csv(path)
#remove nulls
beer = beer[-beer.isna().any(axis=1)]

100%|██████████| 27.4M/27.4M [00:00<00:00, 76.5MB/s]


#### Multiple reviews for the same item
We found earlier that there were around 14000 instances of a user reviewing the same beer more than once. Since basic collaborative filtering frameworks only account for a single user-item interaction, we need to specify an approach for dealing with these cases. In our simple model, we'll take the most recent rating as the "true" value. Later we might experiment with different approaches.

In [92]:
# let's make a new dataframe
beer_simple = beer.copy()
# sort by the relevant columns
beer_simple = beer_simple.sort_values(by=['review_profilename', 'beer_beerid', 'review_time'])
# keep only the most recent review for the user-beer key
beer_simple = beer_simple.drop_duplicates(subset=['review_profilename', 'beer_beerid'], keep="last")


In [93]:
# test using SQL
query = "SELECT review_profilename, beer_beerid \
    FROM beer_simple GROUP BY review_profilename, beer_beerid\
    HAVING COUNT(*)>1 \
    ORDER BY review_profilename, beer_beerid"
#use duckdb to query the data
db.sql(query).df()


,review_profilename,beer_beerid


#### Threshold Choice
We're going to look at the performance of models using several different thresholds for review counts. There are some different considerations to make. First of all, we saw from the EDA that many beers and users only have one review - this is the cold start problem. To construct a meaningful collaborative filter model, we'll need at least three reviews per user/item. In the special case of using 3 as a threshold, we'll have to forgo the validation set entirely so that we have multiple data points per user/item. We'll investigate how different thresholds affect the tradeoff between coverage of recommended items and the quality of recommendations.

As a baseline, we're going to start with a requiring at least 5 reviews per user and 3 reviews per item. These thresholds have been chosen since we want to balance allowing the model to recommend a large amount of items (less strict item threshold) while providing high-quality recommendations (stricter user threshold). Later, we'll experiment with different thresholds.

In [94]:
#create a dataframe for users and beers with the specific threshold
# keep only relevant columns
baseline = beer_simple[['review_profilename', 'beer_beerid', 'review_time']]
baseline = beer_simple.groupby('review_profilename').filter(lambda x: x.shape[0] >= 5)
baseline = baseline.groupby('beer_beerid').filter(lambda x: x.shape[0] >= 3)

In [95]:
beer_simple.nunique().loc[['review_profilename','beer_beerid', 'beer_style']]

review_profilename    32908
beer_beerid           49000
beer_style              104
dtype: int64

In [96]:
baseline.nunique().loc[['review_profilename','beer_beerid', 'beer_style']]

review_profilename    14602
beer_beerid           25851
beer_style              104
dtype: int64

In this case, we see that we've retained around 52% of items. As our model is quite simple, we'll lose a lot of coverage (almost half of all items). To properly address this, we would need to expand our model (e.g. using content-based recommendations with NLP), but since this is a simple project, we'll proceed.

#### Data Splitting
Now it's time to split our data. We're going to leave the last rating as a test - we'll try and predict a user's *next* rating using all their past ratings as training data. This data splitting method approximates many real-world use cases, where we might want to predict a user's future behaviour given their actions until the current time.

In [97]:
%%capture
# generate test set and update training set
# save the last review for each user
test = baseline.drop_duplicates(subset=['review_profilename'], keep="last")
# remove last review in dataframe
train = baseline.groupby('review_profilename', group_keys=False).apply(
    lambda x: x.iloc[:-1])

In [98]:
%%capture
# generate validation set and update training set
# save the last review for each user
validation = train.drop_duplicates(subset=['review_profilename'], keep="last")
# remove last review in dataframe
train = train.groupby('review_profilename', group_keys=False).apply(
    lambda x: x.iloc[:-1])

In [99]:
# test that we've split correctly
baseline.shape[0] == train.shape[0] + validation.shape[0] + test.shape[0]

True

#### Formatting our Data for CF
Now we need to make a user-item matrix. Our simple model is only going to use the overall rating data.

In [100]:
# step 1: encode users and items to integer indices
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
# fit encoders to the values in the set
user_encoder.fit(train['review_profilename'])  
item_encoder.fit(train['beer_beerid'])
# create a mapping from original values to integer indices
user_map = dict(zip(user_encoder.classes_, user_encoder.transform(user_encoder.classes_)))
item_map = dict(zip(item_encoder.classes_, item_encoder.transform(item_encoder.classes_)))
# transform the user and item columns in the train set
user_idx = user_encoder.fit_transform(train['review_profilename'])
item_idx = item_encoder.fit_transform(train['beer_beerid'])

# step 2: create the sparse matrix
ratings = train['review_overall'].values
coo = coo_matrix((ratings, (user_idx, item_idx)))
#convert to CSR format for efficiency
ui_csr = coo.tocsr()

In [101]:
# create columns for user and item maps
user_map = dict(zip(user_encoder.classes_, user_encoder.transform(user_encoder.classes_)))
item_map = dict(zip(item_encoder.classes_, item_encoder.transform(item_encoder.classes_)))

validation['user_idx'] = validation['review_profilename'].map(user_map)
validation['item_idx'] = validation['beer_beerid'].map(item_map)

#drop missing values
validation = validation.dropna(subset=['user_idx', 'item_idx']).astype({'user_idx': int, 'item_idx': int})

We're going to mean-centre each user's score to account for the fact that some users tend to be more lenient or harsh. 

In [102]:
# get sum of scores per user
scores = ui_csr.sum(axis=1).A1
# count number of user reviews
counts = np.diff(ui_csr.indptr)
# get mean vector
mean_scores = scores / counts


Now, it's time to compute the similar matrix. We'll use cosine similarity on our mean-centred data (adjusted-cosine similarity).

In [103]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(ui_csr, dense_output=False)

In [104]:
ui_csr[[1,2],0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 0 stored elements and shape (2, 1)>

### Training
We're ready to begin training our model. For this simple example, we'll validate our choice of $k$ nearest neighbours, first defining a prediction function. Note that we're using a prediction function and rounding to the nearest .5 instead of approaching our ratings as a classificaiton problem. We use this approach for simplicity.
#### Predict Function

In [105]:
def predict(user, item, train, similarity, mean_scores, k, clipped=True):
    """
    Predict ratings for the user-item pair using k nearest neighbours, 
    rounded to the nearest .5 and capped in [0,5]. 
    
    Parameters:
    -user: user index for whom to predict ratings
    -item: item index for which to predict ratings
    -train: training data in sparse matrix format
    -similarity: similarity matrix in sparse format
    -mean_scores: mean scores for each user
    -k: number of nearest neighbouts to consider
    
    Returns: ordinal prediction for user-item pair
    """

    #find neighbours of user for item
    nbs = train[:,item].nonzero()[0]
    nbs = nbs[nbs != user] #exclude self
    if nbs.size == 0:
        # no neighbours, return mean score
        return mean_scores[user]
    
    # get ratings and mean-centre them
    ratings = train[nbs,item].toarray().flatten()
    ratings -= mean_scores[nbs]
    # set limit for k
    k = min(k, nbs.size)

    # get similarity scores
    sims = similarity[user, :].toarray().flatten()
    # get similarity scores for neighbours
    sims = sims[nbs]
    # take k-nearest similarities
    sims = sims[np.argsort(sims)[-k:]]
    # get corresponding k-nearest ratings
    ratings = ratings[np.argsort(sims)[-k:]]
    # compute weighted average
    if np.sum(np.abs(sims)) == 0:
        return mean_scores[user]
    weighted_avg = np.dot(sims, ratings) / np.sum(np.abs(sims))
    # recenter
    weighted_avg += mean_scores[user]
    if clipped == False:
        return weighted_avg
    # round to nearest .5
    weighted_avg = np.round(weighted_avg * 2) / 2 
    # clip to [0,5]
    weighted_avg = np.clip(weighted_avg, 0, 5)
    # return prediction
    return weighted_avg

#### Evaluating the effect of neighbourhood size
Now we'll evaluate over different choices of k.

In [106]:
def evaluate_k(train, validation, similarity, mean_scores, k):
    preds = []
    actuals = []

    for row in validation.itertuples(index=False):
        u = row.user_idx
        i = row.item_idx
        true_r = row.review_overall
        pred = predict(u, i, train, similarity, mean_scores, k)
        preds.append(pred)
        actuals.append(true_r)

    return np.sqrt(mean_squared_error(actuals, preds))  # RMSE

In [107]:
# let's experiment with different values of k
k_values = [3, 5, 10, 20, 50, 100]
train, validation, similarity, mean_scores = ui_csr, validation, cos_sim, mean_scores
for k in k_values:
    print(f'The RMSE for user-based CF with {k}-NN is \
          {evaluate_k(train, validation, similarity, mean_scores, k)}')

The RMSE for user-based CF with 3-NN is           0.8292517772308085
The RMSE for user-based CF with 5-NN is           0.7954208555961889
The RMSE for user-based CF with 10-NN is           0.7741529402438432
The RMSE for user-based CF with 20-NN is           0.7671234633811842
The RMSE for user-based CF with 50-NN is           0.7615911347614778
The RMSE for user-based CF with 100-NN is           0.7591672808836532


#### Top-N predictions
We're not only interested in prediction accuracy; we'd also like to know how effective our algorithm is at predicting novel or less popular items. To measure this, we'll look at the top-N items as calculated by taking the $N$ items for a user with the highest predicted ratings. To avoid interminable runtime, we'll use a a fast matrix-based function which precomputes the most similar neighbours for each user (not for a user-item pair).


In [108]:
def predict_top_N_fast(user, train, similarity, mean_scores, k=10, N=10):
    """
    Fast top-N prediction using user-based CF with vectorized matrix ops.

    Parameters:
    - user: target user index
    - train: CSR matrix of shape (n_users, n_items)
    - similarity: (n_users, n_users) sparse matrix or dense array
    - mean_scores: array of user mean ratings
    - k: number of nearest neighbours
    - N: number of top items to return

    Returns:
    - List of top-N item indices predicted for the user
    """
    # 1. Get top-k most similar users to target user
    user_sims = similarity[user, :].toarray().flatten()
    topk_idx = np.argsort(user_sims)[-k:]
    topk_sims = user_sims[topk_idx]  # shape: (k,)

    # 2. Get their ratings and mean-center
    ratings = train[topk_idx, :].toarray()  # shape: (k, n_items)
    means = mean_scores[topk_idx][:, np.newaxis]
    ratings_centered = ratings - means  # shape: (k, n_items)

    # 3. Weighted sum of centered ratings
    numerator = topk_sims @ ratings_centered  # shape: (n_items,)
    denominator = np.sum(np.abs(topk_sims)) + 1e-8  # to avoid div by 0

    preds = mean_scores[user] + numerator / denominator  # shape: (n_items,)

    # 4. Mask out already rated items
    rated_items = train[user, :].nonzero()[1]
    preds[rated_items] = -np.inf  # exclude known ratings

    # 5. Return top-N items
    top_N_items = np.argsort(preds)[-N:][::-1]
    return top_N_items.tolist()

In [109]:
k_values = [3, 5, 10, 20]
N = 10

for k in k_values:
    recommended_beers = set()
    for user in range(ui_csr.shape[0]):
        preds = predict_top_N_fast(user, train, similarity, mean_scores, k, N)
        # update the set of recommended beers
        recommended_beers.update(preds)
    print(f'For k = {k} nearest neighbours and top-{N} beers recommended:')
    print(f'The number of recommended beers over all users is {len(recommended_beers)}')
    print(f'This corresponds to {len(recommended_beers) / ui_csr.shape[1] * 100:.2f}% \
        of all beers in the training matrix.')

For k = 3 nearest neighbours and top-10 beers recommended:
The number of recommended beers over all users is 6488
This corresponds to 25.18%         of all beers in the training matrix.
For k = 5 nearest neighbours and top-10 beers recommended:
The number of recommended beers over all users is 6146
This corresponds to 23.85%         of all beers in the training matrix.
For k = 10 nearest neighbours and top-10 beers recommended:
The number of recommended beers over all users is 5051
This corresponds to 19.60%         of all beers in the training matrix.
For k = 20 nearest neighbours and top-10 beers recommended:
The number of recommended beers over all users is 3822
This corresponds to 14.83%         of all beers in the training matrix.


It's worth mentioning that this experiment does not tell the entire story when it comes to recommending items from a catalog. This simple user-based CF model is only capable of recommending beers which have been reviewed at least three times - this already excludes the many beers which have only been reviewed once or twice. This is in addition to the fact that the system is only likely to recommend a fraction of the total available beers as demonstrated by the above code. We can investigate how recommendations differ given different neighbourhood sizes.

In [113]:
def get_top_N_beers(user, train, similarity, mean_scores, baseline, k = 3, N = 5):
    #predict top 5 beers with 3 neighbours
    preds = predict_top_N_fast(user, train, similarity, mean_scores, k, N)
    # get item dict. mappings
    beer_ids = item_encoder.inverse_transform(preds)
    # get some aggregate statistics
    return baseline[baseline['beer_beerid'].isin(beer_ids)].groupby([
        'beer_beerid', 'beer_name', 'brewery_name', 'beer_style'], group_keys=False).agg(
            {'review_overall': ['mean', 'count', 'std'],
            'beer_abv': ['mean']}
        ).sort_values(by=('review_overall', 'count'), ascending=False)

In [114]:
# set params
user, k, N = 69, 3, 10
get_top_N_beers(user, ui_csr, similarity, mean_scores, baseline, k=k, N=N)

review_overall  \
                                                                                                        mean   
beer_beerid beer_name                    brewery_name                 beer_style                               
276         Sierra Nevada Pale Ale       Sierra Nevada Brewing Co.    American Pale Ale (APA)       4.248130   
665         Anchor Liberty Ale           Anchor Brewing Company       American Pale Ale (APA)       4.096364   
3842        Trappistes Rochefort 6       Brasserie de Rochefort       Belgian Strong Dark Ale       4.138889   
21324       Hop Hog India Pale Ale       Lancaster Brewing Company    American IPA                  3.574132   
21328       Edgar IPA                    Bullfrog Brewery             American IPA                  3.992424   
21329       Portsmouth Winter Weizen     Portsmouth Brewery           Dunkelweizen                  4.270833   
21334       Veldensteiner Original Lager Kaiser Bräu GmbH & Co KG     Munich Helles Lager           3.409091   
21331       Doppelbock                   Baron Brewing                Doppelbock                    4.000000   
76886       Nomad                        Great Divide Brewing Company Czech Pilsener                3.583333   
21346       Fruitesse Kriekbier          Brouwerij Liefmans           Fruit / Vegetable Beer        2.333333   

                                                                                                     \
                                                                                              count   
beer_beerid beer_name                    brewery_name                 beer_style                      
276         Sierra Nevada Pale Ale       Sierra Nevada Brewing Co.    American Pale Ale (APA)  2406   
665         Anchor Liberty Ale           Anchor Brewing Company       American Pale Ale (APA)  1375   
3842        Trappistes Rochefort 6       Brasserie de Rochefort       Belgian Strong Dark Ale   756   
21324       Hop Hog India Pale Ale       Lancaster Brewing Company    American IPA              317   
21328       Edgar IPA                    Bullfrog Brewery             American IPA               66   
21329       Portsmouth Winter Weizen     Portsmouth Brewery           Dunkelweizen               24   
21334       Veldensteiner Original Lager Kaiser Bräu GmbH & Co KG     Munich Helles Lager        11   
21331       Doppelbock                   Baron Brewing                Doppelbock                  9   
76886       Nomad                        Great Divide Brewing Company Czech Pilsener              6   
21346       Fruitesse Kriekbier          Brouwerij Liefmans           Fruit / Vegetable Beer      3   

                                                                                                         \
                                                                                                    std   
beer_beerid beer_name                    brewery_name                 beer_style                          
276         Sierra Nevada Pale Ale       Sierra Nevada Brewing Co.    American Pale Ale (APA)  0.529726   
665         Anchor Liberty Ale           Anchor Brewing Company       American Pale Ale (APA)  0.540555   
3842        Trappistes Rochefort 6       Brasserie de Rochefort       Belgian Strong Dark Ale  0.511350   
21324       Hop Hog India Pale Ale       Lancaster Brewing Company    American IPA             0.709359   
21328       Edgar IPA                    Bullfrog Brewery             American IPA             0.515098   
21329       Portsmouth Winter Weizen     Portsmouth Brewery           Dunkelweizen             0.441814   
21334       Veldensteiner Original Lager Kaiser Bräu GmbH & Co KG     Munich Helles Lager      0.735465   
21331       Doppelbock                   Baron Brewing                Doppelbock               0.500000   
76886       Nomad                        Great Divide Brewing Company Czech Pilsener           0.376386   
21346       F

In [115]:
# set params
user, k, N = 69, 20, 10
get_top_N_beers(user, ui_csr, similarity, mean_scores, baseline, k=k, N=N)

review_overall  \
                                                                                                                      mean   
beer_beerid beer_name                        brewery_name                    beer_style                                      
1093        Two Hearted Ale                  Bell's Brewery, Inc.            American IPA                         4.320482   
276         Sierra Nevada Pale Ale           Sierra Nevada Brewing Co.       American Pale Ale (APA)              4.248130   
645         Trappistes Rochefort 10          Brasserie de Rochefort          Quadrupel (Quad)                     4.328133   
14916       Hop Wallop                       Victory Brewing Company         American Double / Imperial IPA       3.987629   
1118        Chocolate Stout                  Rogue Ales                      American Stout                       4.115407   
18862       Burton Baton                     Dogfish Head Brewery            American Double / Imperial IPA       4.010500   
6947        Cuvée Van De Keizer Blauw (Blue) Brouwerij Het Anker             Belgian Strong Dark Ale              4.145920   
3646        Urthel Hibernus Quentum          De Leyerth Brouwerijen (Urthel) Tripel                               4.038462   
27265       Bell's Wheat Love                Bell's Brewery, Inc.            Wheatwine                            3.983553   
35405       Victor's MemoriAle Altbier       Two Brothers Brewing Company    Altbier                              4.095238   

                                                                                                                   \
                                                                                                            count   
beer_beerid beer_name                        brewery_name                    beer_style                             
1093        Two Hearted Ale                  Bell's Brewery, Inc.            American IPA                    2529   
276         Sierra Nevada Pale Ale           Sierra Nevada Brewing Co.       American Pale Ale (APA)         2406   
645         Trappistes Rochefort 10          Brasserie de Rochefort          Quadrupel (Quad)                2083   
14916       Hop Wallop                       Victory Brewing Company         American Double / Imperial IPA  1738   
1118        Chocolate Stout                  Rogue Ales                      American Stout                  1733   
18862       Burton Baton                     Dogfish Head Brewery            American Double / Imperial IPA  1381   
6947        Cuvée Van De Keizer Blauw (Blue) Brouwerij Het Anker             Belgian Strong Dark Ale          723   
3646        Urthel Hibernus Quentum          De Leyerth Brouwerijen (Urthel) Tripel                           221   
27265       Bell's Wheat Love                Bell's Brewery, Inc.            Wheatwine                        152   
35405       Victor's MemoriAle Altbier       Two Brothers Brewing Company    Altbier                           42   

                                                                                                                       \
                                                                                                                  std   
beer_beerid beer_name                        brewery_name                    beer_style                                 
1093        Two Hearted Ale                  Bell's Brewery, Inc.            American IPA                    0.506680   
276         Sierra Nevada Pale Ale           Sierra Nevada Brewing Co.       American Pale Ale (APA)         0.529726   
645         Trappistes Rochefort 10          Brasserie de Rochefort          Quadrupel (Quad)                0.534798   
14916       Hop Wallop                       Victory Brewing Company         American Double / Imperial IPA  0.606045   
1118        Chocolate Stout                  Rogue Ales                      American Stout         

In [117]:
# set params
user, k, N = 420, 3, 10
get_top_N_beers(user, ui_csr, similarity, mean_scores, baseline, k=k, N=N)

review_overall  \
                                                                                                             mean   
beer_beerid beer_name                    brewery_name               beer_style                                      
2093        90 Minute IPA                Dogfish Head Brewery       American Double / Imperial IPA       4.144437   
276         Sierra Nevada Pale Ale       Sierra Nevada Brewing Co.  American Pale Ale (APA)              4.248130   
6108        60 Minute IPA                Dogfish Head Brewery       American IPA                         4.167518   
2512        Chimay Grande Réserve (Blue) Bières de Chimay S.A.      Belgian Strong Dark Ale              4.240586   
672         Chimay Première (Red)        Bières de Chimay S.A.      Dubbel                               4.078561   
622         Smuttynose Robust Porter     Smuttynose Brewing Company American Porter                      4.100462   
3842        Trappistes Rochefort 6       Brasserie de Rochefort     Belgian Strong Dark Ale              4.138889   
1632        Cantillon Kriek 100% Lambic  Brasserie Cantillon        Lambic - Fruit                       3.993256   
1862        Leffe Brune / Brown          Abbaye de Leffe S.A.       Belgian Dark Ale                     3.804217   
1704        Hoegaarden Grand Cru         Brouwerij van Hoegaarden   Belgian Strong Pale Ale              3.951807   

                                                                                                          \
                                                                                                   count   
beer_beerid beer_name                    brewery_name               beer_style                             
2093        90 Minute IPA                Dogfish Head Brewery       American Double / Imperial IPA  3029   
276         Sierra Nevada Pale Ale       Sierra Nevada Brewing Co.  American Pale Ale (APA)         2406   
6108        60 Minute IPA                Dogfish Head Brewery       American IPA                    2349   
2512        Chimay Grande Réserve (Blue) Bières de Chimay S.A.      Belgian Strong Dark Ale         1912   
672         Chimay Première (Red)        Bières de Chimay S.A.      Dubbel                          1432   
622         Smuttynose Robust Porter     Smuttynose Brewing Company American Porter                  866   
3842        Trappistes Rochefort 6       Brasserie de Rochefort     Belgian Strong Dark Ale          756   
1632        Cantillon Kriek 100% Lambic  Brasserie Cantillon        Lambic - Fruit                   519   
1862        Leffe Brune / Brown          Abbaye de Leffe S.A.       Belgian Dark Ale                 332   
1704        Hoegaarden Grand Cru         Brouwerij van Hoegaarden   Belgian Strong Pale Ale          166   

                                                                                                              \
                                                                                                         std   
beer_beerid beer_name                    brewery_name               beer_style                                 
2093        90 Minute IPA                Dogfish Head Brewery       American Double / Imperial IPA  0.564692   
276         Sierra Nevada Pale Ale       Sierra Nevada Brewing Co.  American Pale Ale (APA)         0.529726   
6108        60 Minute IPA                Dogfish Head Brewery       American IPA                    0.519563   
2512        Chimay Grande Réserve (Blue) Bières de Chimay S.A.      Belgian Strong Dark Ale         0.528037   
672         Chimay Première (Red)        Bières de Chimay S.A.      Dubbel                          0.522660   
622         Smuttynose Robust Porter     Smuttynose Brewing Company American Porter                 0.502893   
3842        Trappistes Rochefort 6       Brasserie de Rochefort     Belgian Strong Dark Ale         0.511350   
1632        Cantillon Kriek 100% Lambic  Brasserie C

In [118]:
# set params
user, k, N = 420, 20, 10
get_top_N_beers(user, ui_csr, similarity, mean_scores, baseline, k=k, N=N)

review_overall  \
                                                                                                                       mean   
beer_beerid beer_name                    brewery_name                         beer_style                                      
2093        90 Minute IPA                Dogfish Head Brewery                 American Double / Imperial IPA       4.144437   
4083        Stone Ruination IPA          Stone Brewing Co.                    American Double / Imperial IPA       4.156041   
88          Stone IPA (India Pale Ale)   Stone Brewing Co.                    American IPA                         4.249588   
276         Sierra Nevada Pale Ale       Sierra Nevada Brewing Co.            American Pale Ale (APA)              4.248130   
34          La Fin Du Monde              Unibroue                             Tripel                               4.289840   
6108        60 Minute IPA                Dogfish Head Brewery                 American IPA                         4.167518   
1708        St. Bernardus Abt 12         Brouwerij St. Bernardus NV           Quadrupel (Quad)                     4.324894   
571         Samuel Smith's Oatmeal Stout Samuel Smith Old Brewery (Tadcaster) Oatmeal Stout                        4.236340   
2512        Chimay Grande Réserve (Blue) Bières de Chimay S.A.                Belgian Strong Dark Ale              4.240586   
672         Chimay Première (Red)        Bières de Chimay S.A.                Dubbel                               4.078561   

                                                                                                                    \
                                                                                                             count   
beer_beerid beer_name                    brewery_name                         beer_style                             
2093        90 Minute IPA                Dogfish Head Brewery                 American Double / Imperial IPA  3029   
4083        Stone Ruination IPA          Stone Brewing Co.                    American Double / Imperial IPA  2541   
88          Stone IPA (India Pale Ale)   Stone Brewing Co.                    American IPA                    2426   
276         Sierra Nevada Pale Ale       Sierra Nevada Brewing Co.            American Pale Ale (APA)         2406   
34          La Fin Du Monde              Unibroue                             Tripel                          2372   
6108        60 Minute IPA                Dogfish Head Brewery                 American IPA                    2349   
1708        St. Bernardus Abt 12         Brouwerij St. Bernardus NV           Quadrupel (Quad)                2113   
571         Samuel Smith's Oatmeal Stout Samuel Smith Old Brewery (Tadcaster) Oatmeal Stout                   1940   
2512        Chimay Grande Réserve (Blue) Bières de Chimay S.A.                Belgian Strong Dark Ale         1912   
672         Chimay Première (Red)        Bières de Chimay S.A.                Dubbel                          1432   

                                                                                                                        \
                                                                                                                   std   
beer_beerid beer_name                    brewery_name                         beer_style                                 
2093        90 Minute IPA                Dogfish Head Brewery                 American Double / Imperial IPA  0.564692   
4083        Stone Ruination IPA          Stone Brewing Co.                    American Double / Imperial IPA  0.531444   
88          Stone IPA (India Pale Ale)   Stone Brewing Co.                    American IPA                    0.509952   
276         Sierra Nevada Pale Ale       Sierra Nevada Brewing Co.            American Pale Ale (APA)         0.529726   
34          La Fin Du Monde              Unibroue                   

We can see clearly that increasing the neighbourhood size tends to recommend more popular beers. This effect is commonly observed when implementing user-based CF. We can think critically about why this may be. When generating recommendations from a small number of neighbours, we only take the recommendations from the most similar users - and their "votes" towards recommendations have a very large impact on the predicted items. Therefore, a more rare and polarizing beer may be recommended to a user when using a small value for k if that user's closest neighbours give it a high score. Conversely, including a larger number of neighbours when generating predictions tends to push predicted scores towards the mean; more rare and polarizing beers will tend to get pushed out by popular and highly-rated beers. The "signal" from similar users will be lost. As we saw from the EDA, there is a relatively small number of highly-rated and highly-reviewed beers. Continuing to increase the value of k will likely lead to those beers being recommended:

In [ ]:
# set params, make k big
user, k, N = 420, 150, 10
get_top_N_beers(user, ui_csr, similarity, mean_scores, baseline, k=k, N=N)

review_overall  \
                                                                                                                                            mean   
beer_beerid beer_name                           brewery_name                                       beer_style                                      
2093        90 Minute IPA                       Dogfish Head Brewery                               American Double / Imperial IPA       4.144437   
412         Old Rasputin Russian Imperial Stout North Coast Brewing Co.                            Russian Imperial Stout               4.169433   
88          Stone IPA (India Pale Ale)          Stone Brewing Co.                                  American IPA                         4.249588   
276         Sierra Nevada Pale Ale              Sierra Nevada Brewing Co.                          American Pale Ale (APA)              4.248130   
34          La Fin Du Monde                     Unibroue                                           Tripel                               4.289840   
6108        60 Minute IPA                       Dogfish Head Brewery                               American IPA                         4.167518   
1708        St. Bernardus Abt 12                Brouwerij St. Bernardus NV                         Quadrupel (Quad)                     4.324894   
131         Ayinger Celebrator Doppelbock       Privatbrauerei Franz Inselkammer KG / Brauerei ... Doppelbock                           4.293592   
571         Samuel Smith's Oatmeal Stout        Samuel Smith Old Brewery (Tadcaster)               Oatmeal Stout                        4.236340   
141         Hennepin (Farmhouse Saison)         Brewery Ommegang                                   Saison / Farmhouse Ale               4.243311   

                                                                                                                                         \
                                                                                                                                  count   
beer_beerid beer_name                           brewery_name                                       beer_style                             
2093        90 Minute IPA                       Dogfish Head Brewery                               American Double / Imperial IPA  3029   
412         Old Rasputin Russian Imperial Stout North Coast Brewing Co.                            Russian Imperial Stout          2892   
88          Stone IPA (India Pale Ale)          Stone Brewing Co.                                  American IPA                    2426   
276         Sierra Nevada Pale Ale              Sierra Nevada Brewing Co.                          American Pale Ale (APA)         2406   
34          La Fin Du Monde                     Unibroue                                           Tripel                          2372   
6108        60 Minute IPA                       Dogfish Head Brewery                               American IPA                    2349   
1708        St. Bernardus Abt 12                Brouwerij St. Bernardus NV                         Quadrupel (Quad)                2113   
131         Ayinger Celebrator Doppelbock       Privatbrauerei Franz Inselkammer KG / Brauerei ... Doppelbock                      2013   
571         Samuel Smith's Oatmeal Stout        Samuel Smith Old Brewery (Tadcaster)               Oatmeal Stout                   1940   
141         Hennepin (Farmhouse Saison)         Brewery Ommegang                                   Saison / Farmhouse Ale          1794   

                                                                                                                                             \
                                                                                                                                        std   
beer_beerid beer_name                           brewery_name                                       beer_style  

All of these beers are among the most popular and well-liked beers on the site. This demonstrates why it's not beneficial to choose a large value of k simply because it results in a low RMSE; in the context of an item catalog, a recommender should be capable of providing novel recommendations.